In [1]:
from sifce import datatools, population, network
import pandas as pd
import numpy as np
from bilby.core.prior import Uniform, Sine, Cosine, PowerLaw, Constraint, DeltaFunction
from pycbc.detector import Detector
from bilby.core.prior import PriorDict
from pycbc.psd import analytical
pd.set_option('display.max_columns', None)

In [2]:
net = network.Network(ifos=["H1", "L1"])

In [3]:
sample_rate=1024
flow = 20.0
delta_f = 1.0/8
flen = int(sample_rate/delta_f)
psd = analytical.aLIGOZeroDetHighPower(flen, delta_f, flow)

net.network_psds = {'H1': psd, 'L1': psd, 'V1':psd}

In [4]:
prior_gw150914 = dict(mass_1= DeltaFunction(35),
                     mass_2= DeltaFunction(35),
                     a_1 = DeltaFunction(0.5),
                     a_2 = DeltaFunction(0.5),
                     tilt_1 = DeltaFunction(1.4),
                     tilt_2 = DeltaFunction(1.4),
                     phi_12 = DeltaFunction(3.09),
                     phi_jl = DeltaFunction(5.21),
                     theta_jn =  Sine(name='theta_jn'),
                     phase = DeltaFunction(1.04),
                     reference_frequency = DeltaFunction(20),
                     dec=Cosine(name='dec'),
                     ra=Uniform(name='ra', minimum=0, maximum=2 * np.pi, boundary='periodic'),
                     psi =  Uniform(name='psi', minimum=0, maximum=np.pi, boundary='periodic'),
                     luminosity_distance = PowerLaw(alpha=2, name='luminosity_distance', minimum=50, maximum=2000, unit='Mpc', latex_label='$d_L$'))

In [5]:
prior_obj = PriorDict(dictionary = prior_gw150914)
pop = population.SimulationSet(distribution=prior_obj)
pop.sample_distribution(100)

In [6]:
for i in range(len(pop.simulations_dataframe.index)):
    row = pop.simulations_dataframe.iloc[[0]]
    pop.calc_snrs_sky(row, net, N=10)

In [7]:
for i in range(len(pop.simulations_dataframe.index)):
    row = pop.simulations_dataframe.iloc[[0]]
    pop.sample_distances(row, net, N=10)

In [8]:
pop.simulations_dataframe

,mass_1,mass_2,spin_1x,spin_1y,spin_1z,spin_2x,spin_2y,spin_2z,theta_jn,phase,ra,dec,psi,geocent_time,luminosity_distance,redshift,chirp_mass,total_mass,mass_ratio,symmetric_mass_ratio,mass_1_source,mass_2_source,chirp_mass_source,total_mass_source,a_1,a_2,tilt_1,tilt_2,phi_12,phi_jl,reference_frequency,cal_idx,opt_snr,template_snr,iota,phi_1,phi_2
0,35.0,35.0,0.428327,-0.243545,0.084984,-0.415197,0.265310,0.084984,1.425536,1.04,1.383118,0.538646,2.614157,NaN,876.581341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,10.350622,NaN,1.430871,5.766177,2.572991
1,35.0,35.0,0.428327,-0.243545,0.084984,-0.415197,0.265310,0.084984,1.425536,1.04,1.383118,0.538646,2.614157,NaN,1963.428088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,4.621082,NaN,1.430871,5.766177,2.572991
2,35.0,35.0,0.428327,-0.243545,0.084984,-0.415197,0.265310,0.084984,1.425536,1.04,1.383118,0.538646,2.614157,NaN,1437.912785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,6.309953,NaN,1.430871,5.766177,2.572991
3,35.0,35.0,0.428327,-0.243545,0.084984,-0.415197,0.265310,0.084984,1.425536,1.04,1.383118,0.538646,2.614157,NaN,1372.682327,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,6.609805,NaN,1.430871,5.766177,2.572991
4,35.0,35.0,0.428327,-0.243545,0.084984,-0.415197,0.265310,0.084984,1.425536,1.04,1.383118,0.538646,2.614157,NaN,941.919386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,9.632631,NaN,1.430871,5.766177,2.572991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,35.0,35.0,0.428773,-0.242759,0.084984,-0.415683,0.264547,0.084984,2.022885,1.04,0.588309,0.866484,0.188713,NaN,1822.287402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,16.272527,NaN,2.028218,5.768011,2.574826
9996,35.0,35.0,0.428773,-0.242759,0.084984,-0.415683,0.264547,0.084984,2.022885,1.04,0.588309,0.866484,0.188713,NaN,1991.943721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,14.886576,NaN,2.028218,5.768011,2.574826
9997,35.0,35.0,0.428773,-0.242759,0.084984,-0.415683,0.264547,0.084984,2.022885,1.04,0.588309,0.866484,0.188713,NaN,1954.030390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,15.175415,NaN,2.028218,5.768011,2.574826
9998,35.0,35.0,0.428773,-0.242759,0.084984,-0.415683,0.264547,0.084984,2.022885,1.04,0.588309,0.866484,0.188713,NaN,1923.476603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,15.416471,NaN,2.028218,5.768011,2.574826


In [9]:
from astropy.cosmology import Planck18
tv = pop.sensitive_volume_calc(11, Planck18)

In [10]:
tv

<Quantity 3.07492772 Gpc3 yr>